- Importação das bibliotecas
- Definição da variável data

In [ ]:
import datetime
import pyspark.sql.functions as F
date = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")

In [ ]:
dfDepartment = spark.read.json("s3://datalake01-raw/ADV_AULA_DBO_DEPARTMENT")
dfTemp = dfDepartment.select("DepartmentID", "Name")

In [ ]:
dfTemp.show()

In [ ]:
dfDepartment.describe().show()

In [ ]:
dfTemp.orderBy(dfTemp["DepartmentID"].desc()).show(5)

In [ ]:
dfTemp.count()

In [ ]:
dfTemp.select("DepartmentID").groupBy("DepartmentID").count().orderBy("DepartmentID").show()


In [ ]:
dfTempSql = spark.sql("""select DepartmentID, count(*) 
                         from datalake01_refined.adv_aula_dbo_department 
                         group by DepartmentID 
                         order by DepartmentID""")
dfTempSql.show()

- DEPARTMENT

In [ ]:
dfDepartment = spark.read.json("s3://datalake01-raw/ADV_AULA_DBO_DEPARTMENT")
dfDepartment.printSchema()

In [ ]:
dfDepartment = spark.read.json("s3://datalake01-raw/ADV_AULA_DBO_DEPARTMENT/")
#df.show()
primary_key = dfDepartment.select("PrimaryKey").first()[0]
#print(primary_key)
dfAggDepartment = (dfDepartment.groupBy(primary_key).agg(F.max("ModifiedDate").alias("ModifiedDate")))
dfAggDepartment.sort("DepartmentID").show()

In [ ]:
pathDepartment = "s3://datalake01-refined/ADV_AULA_DBO_DEPARTMENT/date_partition=" + date
dfDepartment.join(dfAggDepartment, dfAggDepartment.columns).coalesce(1).write.mode("append").parquet(pathDepartment)

- EMPLOYEE

In [ ]:
dfEmployee = spark.read.json("s3://datalake01-raw/ADV_AULA_DBO_EMPLOYEE")
dfEmployee.printSchema()

In [ ]:
dfEmployee = spark.read.json("s3://datalake01-raw/ADV_AULA_DBO_EMPLOYEE/")
#df.show()
primary_key = dfEmployee.select("PrimaryKey").first()[0]
#print(primary_key)
dfAggEmployee = (dfEmployee.groupBy(primary_key).agg(F.max("ModifiedDate").alias("ModifiedDate")))
dfAggEmployee.sort("BusinessEntityID").show()

In [ ]:
pathEmployee = "s3://datalake01-refined/ADV_AULA_DBO_EMPLOYEE/date_partition=" + date
dfEmployee.join(dfAggEmployee, dfAggEmployee.columns).coalesce(1).write.mode("append").parquet(pathEmployee)

- PERSON

In [ ]:
dfPerson = spark.read.json("s3://datalake01-raw/ADV_AULA_DBO_PERSON")
dfPerson.printSchema()

In [ ]:
dfPerson = spark.read.json("s3://datalake01-raw/ADV_AULA_DBO_PERSON/")
#df.show()
primary_key = dfPerson.select("PrimaryKey").first()[0]
#print(primary_key)
dfAggPerson = (dfPerson.groupBy(primary_key).agg(F.max("ModifiedDate").alias("ModifiedDate")))
dfAggPerson.sort("BusinessEntityID").show()

In [ ]:
pathPerson = "s3://datalake01-refined/ADV_AULA_DBO_PERSON/date_partition=" + date
dfPerson.join(dfAggPerson, dfAggPerson.columns).coalesce(1).write.mode("append").parquet(pathPerson)

- EMPLOYEE DEPARTMENT HISTORY

In [ ]:
dfHistory = spark.read.json("s3://datalake01-raw/ADV_AULA_DBO_EMPLOYEEDEPARTMENTHISTORY")
dfHistory.printSchema()

In [ ]:
dfHistory = spark.read.json("s3://datalake01-raw/ADV_AULA_DBO_EMPLOYEEDEPARTMENTHISTORY/")
#df.show()
primary_key = dfHistory.select("PrimaryKey").first()[0]
#print(primary_key)
dfAggHistory = (dfHistory.groupBy(primary_key).agg(F.max("ModifiedDate").alias("ModifiedDate")))
dfAggHistory.sort("BusinessEntityID").show()

In [ ]:
pathHistory = "s3://datalake01-refined/ADV_AULA_DBO_EMPLOYEEDEPARTMENTHISTORY/date_partition=" + date
dfHistory.join(dfAggHistory, dfAggHistory.columns).coalesce(1).write.mode("append").parquet(pathHistory)

- SPARK SQL

In [ ]:
dfSQL = spark.sql("""SELECT 
                    e.BusinessEntityID
                    ,p.Title
                    ,p.FirstName
                    ,p.MiddleName
                    ,p.LastName
                    ,p.Suffix
                    ,e.JobTitle
                    ,d.Name AS Department
                    ,d.GroupName
                    ,edh.StartDate 
                FROM  datalake01_refined.adv_aula_dbo_employee  e
                    INNER JOIN datalake01_refined.adv_aula_dbo_person  p
                    ON p.BusinessEntityID = e.BusinessEntityID
                    INNER JOIN datalake01_refined.adv_aula_dbo_employeedepartmenthistory  edh 
                    ON e.BusinessEntityID = edh.BusinessEntityID
                    INNER JOIN datalake01_refined.adv_aula_dbo_department d 
                    ON edh.DepartmentID = d.DepartmentID
                WHERE edh.EndDate IS NULL""")
dfSQL.show()